In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data 
import time

In [2]:
physicsClient = p.connect(p.GUI)

In [3]:
p.setGravity(0,0,-10) 
p.resetSimulation() 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) 
planeId = p.loadURDF("plane.urdf") 
robotId = p.loadURDF("iiwa7.urdf",flags=9, useFixedBase=1)

robotStartPos = [0,0,0]
robotStartOrientation = p.getQuaternionFromEuler([0,0,0])

p.resetBasePositionAndOrientation(robotId,robotStartPos,robotStartOrientation)

p.setJointMotorControlArray(robotId,range(7),p.VELOCITY_CONTROL,forces=np.zeros(7))

In [4]:
a=1
horizon_length = 1000
Q2=0.1*np.eye(14)
Q1=1000*np.eye(14)
for i in range(7,14):
    Q1[i,i]=0.1
Q=[]
for i in range(1001):
    if (i==1000):
        Q.append(Q1)
    else:
        Q.append(Q2)

R=0.01*np.eye(7)
sigma=0.000001
C=np.eye(14)
Sigma=np.eye(14)


u_bar = np.zeros([7, horizon_length])
U = np.zeros([7, horizon_length])
Z = np.zeros([14,1001])
u= np.zeros([7, horizon_length])
    
      
z_bar=np.zeros([14,1001])
target_p = np.array([1,1,1])
robotEndOrientation = p.getQuaternionFromEuler([0,0,0])
targetPositionsJoints = p.calculateInverseKinematics(robotId,7,target_p,targetOrientation = robotEndOrientation)
p.setJointMotorControlArray(robotId,range(7),p.POSITION_CONTROL,targetPositions = targetPositionsJoints)
z_bar[0:7,1000]=targetPositionsJoints
#for i in range (100):
#    p.stepSimulation()
#for i in range(7):
#    z_bar[i,1000]=p.getJointStates(robotId,range(7))[i][0]
#for i in range(7):
#    z_bar[i+7,1000]=p.getJointStates(robotId,range(7))[i][1]


In [5]:
def simulate_system(x, u):
    x_next=[]
    for i in range(7):
        p.resetJointState(robotId,i,x[i],targetVelocity = x[i+7])
    
    p.setJointMotorControlArray(robotId,range(7), controlMode=p.TORQUE_CONTROL,forces=u)
    p.stepSimulation()
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][0])
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][1])
    x_next = np.array(x_next)
    return x_next
    
    

In [6]:
def get_linearization(current_state, control_signal):
    eps = 1e-5
    A = np.zeros([len(current_state), len(current_state)])
    for ii in range(len(current_state)):
        x = current_state.copy()
        x[ii] += eps
        x_inc = simulate_system(x, control_signal)  
        x = current_state.copy()
        x[ii] -= eps
        x_dec = simulate_system(x, control_signal)
        A[:,ii] = (x_inc - x_dec) / (2 * eps)
 
    B = np.zeros([len(current_state), len(control_signal)])
    for ii in range(len(control_signal)):
        u = control_signal.copy()
        u[ii] += eps
        x_inc = simulate_system(current_state, u)
        u = control_signal.copy()
        u[ii] -= eps
        x_dec = simulate_system(current_state, u)
        B[:,ii] = (x_inc - x_dec) / (2 * eps)
    return A, B

In [7]:
def solve_ricatti_equations(Z,U,Q,R,N,q,r):
    
    
    K_gains = []
    k_feedforward = []
    P = Q[-1]
    p = q[-1].T
       
    for i in reversed(range(horizon_length)):
        A, B = get_linearization(Z[:,i], U[:,i])
        K = -np.linalg.inv(B.T @ P @ B + R) @ B.T @ P @ A
        K_gains.append(K)
        k = -np.linalg.inv(B.T @ P @ B + R) @ (B.T @ p + r[i].T)
        k_feedforward.append(k)
        W = np.linalg.inv(Sigma)-sigma*C.T.dot(P).dot(C)
        iW = np.linalg.inv(W)
        p = q[i].T + A.T @ p + A.T @ P @ B @ k + sigma * P.T @ C @ iW @ C.T @ p
        P = Q[i] + A.T @ P @ A + A.T @ P @ B @ K + sigma * P.T @ C @ iW @ C.T @ P
       
       
    K_gains = K_gains[::-1]
    k_feedforward = k_feedforward[::-1]
    return K_gains, k_feedforward

In [8]:
def controller(Z,U,q,r,horizon_length,Q,R):
    state2=np.empty([14, horizon_length+1])
    z0 = np.zeros([14,])
    state2[:,0] = z0
    u2 = np.zeros([7, 1000])
    K,k = solve_ricatti_equations(Z,U,Q,R,horizon_length,q,r)
    for i in range(horizon_length):
        u2[:,i] = U[:,i] + K[i] @ (state2[:,i] - Z[:,i])+ a*k[i]
        state2[:,i+1] = simulate_system(state2[:,i], u2[:,i])
        
    
    state=state2.copy()
    u=u2.copy()
    return state,u

In [9]:
def compute_cost(Z,U,z_bar,u_bar, horizon_length):
    J=((Z[:,horizon_length]-z_bar[:,horizon_length]).T)@ Q[horizon_length] @(Z[:,horizon_length]-z_bar[:,horizon_length])
    for i in range(horizon_length):
        J=J+((Z[:,i]-z_bar[:,i]).T)@ Q[i] @(Z[:,i]-z_bar[:,i])+((U[:,i]-u_bar[:,i]).T @ R @ (U[:,i]-u_bar[:,i]))
    
    return J

In [10]:
def get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar):
    q=[]
    r=[]
    J=compute_cost(Z,U,z_bar,u_bar, horizon_length)
    #J=J+0.5*((z[:,horizon_length]-Z[:,horizon_length]).T)@ Q[horizon_length] @(z[:,horizon_length]-Z[:,horizon_length])+2*(z[:,horizon_length].T-z_bar[:,horizon_length].T)@Q[horizon_length]@(z[:,horizon_length]-Z[:,horizon_length])
    for i in range(horizon_length):
        #J=J+0.5*((u[:,i]-U[:,i]).T)@ R @(u[:,i]-U[:,i])+0.5*((z[:,i]-Z[:,i]).T)@ Q[i] @(z[:,i]-Z[:,i])+2*((z[:,i]-z_bar[:,i]).T)@Q[i]@(z[:,i]-Z[:,i])+2*((u[:,i]-u_bar[:,i]).T)@R@(u[:,i]-U[:,i])    
        q1=2*(Z[:,i]-z_bar[:,i]).T@Q[i]
        q.append(q1)
        r1=2*(U[:,i]-u_bar[:,i]).T@R
        r.append(r1)
    
    return J,q,r

In [11]:
J,q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
state,u = controller(Z,U,q,r,horizon_length,Q,R)
U = u
Z = np.zeros([14, horizon_length+1])
for i in range(horizon_length):
    Z[:,i+1] = simulate_system(Z[:,i], U[:,i])
J1=J
print(J)

while 1:
    J,q,r=get_quadratic_approximation_cost(Z,U, horizon_length,z_bar,u_bar)
    state,u = controller(Z,U,q,r,horizon_length,Q,R)
    print(J)
    if (J<=J1):
        U = u
        Z = np.zeros([14, horizon_length+1])
        for i in range(horizon_length):
            Z[:,i+1] = simulate_system(Z[:,i], U[:,i])
        if (abs(J1-J)<0.05):
            print("iteration converged")
            break
        J1=J
    else:
        a=a/2
        J1=J
        print(a)
        
            
    
    

3867.7820315414046
1555.485844738335
1555.486022160221
0.5
1555.486022160221
iteration converged


In [12]:
x = np.zeros([14,1])
for i in range(7):
    p.resetJointState(robotId,i,x[i],targetVelocity = x[i+7])

In [13]:
for i in range(1000):
    p.setJointMotorControlArray(robotId,range(7), controlMode=p.TORQUE_CONTROL,forces= u[:,i])
    p.stepSimulation()